<a href="https://colab.research.google.com/github/viksingh/TrendFollowingStrategies/blob/master/100_day_weekly_with_weekly_snapshot_27_01_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import glob
import os
import numpy as np
import sys
import time
#import pdb
start_time = time.time()

from copy import deepcopy
from google.colab import drive
drive.mount('/content/drive')
!{sys.executable} -m pip install PyDrive
!{sys.executable} -m pip install yfinance


import yfinance as yf
import yfinance as yft

# Check if index is going up

moneyforeachstock = 2500
amountSpent = 0
c_stoplossfactorindexdown = 0.00
c_stoplossindexup = 0.00
numberOfStocks = 0
totalStocksHeld = 0

overAllStocksPurchased = 0
moneySpentForLosingStocks = 0

totalSpent = 0
totalValue = 0
totalStopLossMoney = 0
indexPath = '/content/drive/My Drive/Colab Notebooks/data2/index.csv'
stockCombinedPath = '/content/drive/My Drive/Colab Notebooks/data2/stockscombined.csv'
currentPricePath = '/content/drive/My Drive/Colab Notebooks/data2/CurrentPrice.csv'

dfindex = pd.read_csv(indexPath, header=0)
dfstocks = pd.read_csv(stockCombinedPath, header=0 )
dfCurrentPrice = pd.read_csv(currentPricePath, header=0 )

# Create a dataframe to store currently bought stocks
# Fields : STOCK, BOUGHT DATE , BOUGHT PRICE , QTY, UPDATE DATE, LAST PRICE, RoC % , BOUGHTPRICE , UPDTPRICE
# May be add an entry for each week when this gets checked

def fround(f):
    return str(round(f,2))

def createqueryDate(d):
    return 'DATE' + '==' + "'" + d + "'"

def checkActive():
    return "ACTIVE == 'X'"

def checkStock(s):
     return 'STOCK' + '==' + "'" + s + "'"

#def get_Current_stockPrice(s,df):
#%load_ext google.colab.data_table

#pdb.set_trace()

from datetime import date, timedelta

start_date = date(2019, 9, 6)
end_date = date(2019, 12, 27)

start_date_print = deepcopy(start_date)
end_date_print = deepcopy(end_date)

delta = timedelta(days=28)


ENDDATE = end_date.strftime("%Y-%m-%d")
companiesList = dfstocks.STOCK.unique()

dfStocksCurrentlyBought = pd.DataFrame(columns=['STOCK','PURCHASEDATE','PURCHASEPRICE','QTY','UPDATEDATE','UPDATEPRICE','MAXPRICE','ROC', 'ACTIVE'])

while start_date <= end_date:
    weektotalValue = 0
    weektotalValueAPI = 0
# Check if index is increasing / decreasing this week
    processingDate = start_date.strftime("%Y-%m-%d")
    dfNasdaqCheckIndexIncreasing = dfindex.query(createqueryDate(processingDate))
    if (dfNasdaqCheckIndexIncreasing.shape[0])> 0:
        isIndexIncreasing = dfNasdaqCheckIndexIncreasing['CROSS'].iloc[-1]
        print('Index : ' + str(processingDate) + ' - ' + str(isIndexIncreasing))

# Update to current weeks price

        stockWeekUpdatePriceList = dfStocksCurrentlyBought.query("ACTIVE=='X'").STOCK.unique()
        for stockUpdatePrice in stockWeekUpdatePriceList:
            queryGetCurrentStockUpdatePrice = 'STOCK' + '==' + "'" + stockUpdatePrice + "'"
            dfCurrentstockUpdatePrice = dfstocks.query(queryGetCurrentStockUpdatePrice)

            queryGetUpdatePriceDateValue = "DATE" + "==" + "'" + processingDate + "'"
            dfCurrentstockUpdatePrice = dfCurrentstockUpdatePrice.query(queryGetUpdatePriceDateValue)
            dfCurrentstockUpdatePrice = dfCurrentstockUpdatePrice.tail(1)

            # SUMMARY
            if (dfCurrentstockUpdatePrice.shape[0]) > 0:
                priceForWeek = float(dfCurrentstockUpdatePrice['PRICE'].iloc[0])
                dqQty = dfStocksCurrentlyBought.query(queryGetCurrentStockUpdatePrice)
                weekQtyForUpdate = float(dqQty['QTY'].iloc[0])
                maxPriceTillNow = float(dqQty['MAXPRICE'].iloc[0])
                weekQtyForUpdate = float(dqQty['QTY'].iloc[0])
                updatePurchasePrice = float(dqQty['PURCHASEPRICE'].iloc[0])
                updateCurrentPrice = float(dqQty['CURRPRICE'].iloc[0])
                updateCurrentValue = float(dqQty['CURRVALUE'].iloc[0])

                if( maxPriceTillNow < priceForWeek):
                    maxPriceTillNow = priceForWeek
                    dfStocksCurrentlyBought = dfStocksCurrentlyBought.append({'STOCK': stockUpdatePrice, 'PURCHASEDATE': processingDate, 'PURCHASEPRICE': updatePurchasePrice, 'QTY': weekQtyForUpdate, 'CURRVALUE': updateCurrentValue, 'CURRPRICE': updateCurrentPrice, 'MAXPRICE': maxPriceTillNow, 'ROC': ROI, 'ACTIVE': 'X'}, ignore_index=True)



######################----------------------###################################
######## SELL STOCK LOGIC #####################################################
######################----------------------###################################
        dfstocksActive = dfStocksCurrentlyBought.query(checkActive())
        stockBoughtList = dfstocksActive.STOCK.unique()

        for stockToSellCheck in stockBoughtList:

            dfStockWithCurrentDate = dfstocks.query(createqueryDate(processingDate))
            queryGetCurrentPriceWithDateSTOCK = checkStock(stockToSellCheck)
            dfStockWithCurrentDate = dfStockWithCurrentDate.query(queryGetCurrentPriceWithDateSTOCK)
# SELLING RULES
# 1)If increasing index this week : current price is higher than 0.8 MAX PRICE => sell
# 2) If currennt price or max price is lowe than 0.9 max price => sell
            listMaxStockPrice = dfStocksCurrentlyBought.groupby(['STOCK'])['MAXPRICE'].max()
            stockPriceThisStock =  round(float (listMaxStockPrice.get(key=stockToSellCheck)),2)
            # Get current price if it's less than 0.8 MAX PRICE if index increasing or less than 0.9 index decreasing
            if dfStockWithCurrentDate.size > 0:
                currentStockPrice = dfStockWithCurrentDate.iloc[-1]['PRICE']
                dfCurrentStockData = dfStocksCurrentlyBought.query(queryGetCurrentPriceWithDateSTOCK )
                STOCKQTY = dfCurrentStockData.iloc[-1]['QTY']
                if (isIndexIncreasing ==1):
                    stopLossPrice = c_stoplossfactorindexdown *  stockPriceThisStock
                else:
                    stopLossPrice = c_stoplossindexup *  stockPriceThisStock

                if (currentStockPrice < stopLossPrice):
                    moneySpentForLosingStocks = moneySpentForLosingStocks + dfCurrentStockData.iloc[-1]['PURCHASEPRICE'] * STOCKQTY
                    print('Money spent for losting stocks' + str(moneySpentForLosingStocks ))
                    dfStocksCurrentlyBought.loc[ (dfStocksCurrentlyBought.STOCK == stockToSellCheck  ),'ACTIVE'] = ''
                    print(" --: STOCK VALUE IS : " + str(stockToSellCheck) +' - ' + str(processingDate) +' - ' +  ' Total Amount : ' + fround(currentStockPrice * STOCKQTY ))
                    totalStocksHeld = totalStocksHeld - 1
                    totalStopLossMoney = totalStopLossMoney + currentStockPrice * STOCKQTY
                    print("STOP LOSS TRIGGERED FOR " + stockToSellCheck + ' .GETTING ' + fround(currentStockPrice * STOCKQTY) + ' AND TOTAL FROM STOP LOSS IS ' + fround(totalStopLossMoney ))

## BUY RULES

    if (isIndexIncreasing == 1):
        for  stock in companiesList :
            queryGetCurrentStockData = 'STOCK' + '==' + "'" + stock + "'"
            dfCurrentStock = dfstocks.query(queryGetCurrentStockData)
            periodToLookBack = 15 # 15 weeks
            queryDateForCurrentStock = "DATE" + "<" + "'" + processingDate + "'"
            dfCurrentStocklookBackData = dfCurrentStock.query(queryDateForCurrentStock)
            dfCurrentStocklookBackData = dfCurrentStocklookBackData.tail(periodToLookBack)
            if dfCurrentStocklookBackData.shape[0] > periodToLookBack - 1:
                start = 0
                length = periodToLookBack - 1
                stockHighestPointValid = True
                latestpriceIndex = length - start
                for n in range(start, start + length):
                    currentweekPrice  = dfCurrentStocklookBackData['PRICE'].iloc[latestpriceIndex]
                    lastweekPrice = dfCurrentStocklookBackData['PRICE'].iloc[n]
                    if (lastweekPrice > currentweekPrice):
                        stockHighestPointValid = False
                        break

                if( stockHighestPointValid == True ):
                    STOCK = dfCurrentStocklookBackData['STOCK'].iloc[-1]
                    VOLUME = dfCurrentStocklookBackData['VOLUME'].iloc[-1]
                    DATE = processingDate #dfCurrentStocklookBackData['DATE'].iloc[-1]
                    ROI = dfCurrentStocklookBackData['ROC100'].iloc[-1]
## how to use this .it should be the same as last price
                    maxPrice = dfCurrentStocklookBackData.loc[dfCurrentStocklookBackData['PRICE'].idxmax()]['PRICE']

                    if (ROI > 30 and ROI < 80 and VOLUME > 1000000 and maxPrice > 15 and stockHighestPointValid == True):

                        dfCheckStockALreadyAdded = dfStocksCurrentlyBought.query(queryGetCurrentStockData)
                        if dfCheckStockALreadyAdded.size == 0:
                            totalStocksHeld = totalStocksHeld + 1
                            overAllStocksPurchased = overAllStocksPurchased + 1
                            CURRENTPRICESTOCKDATE = ''
                            numberOfStocks = int(moneyforeachstock / maxPrice)
                            amountSpent = numberOfStocks *  maxPrice
#                            try:
#                                currentPriceAPICall = str(fround(yf.Ticker(STOCK).info['regularMarketPrice']))
                            queryGetCurrentStockPrice = 'STOCK' + '==' + "'" + stock + "'"
                            dfCurrentStockPrice = dfCurrentPrice.query(queryGetCurrentStockPrice)
                            if(not dfCurrentStockPrice.empty):
                                currentPriceAPICall = float( dfCurrentStockPrice['CURRPRICE'].iloc[0] )
                                currentValueAPICall = fround( currentPriceAPICall * numberOfStocks)
                                print( str(totalStocksHeld) +' ++ : ' + STOCK + ',' + str(processingDate) + ',' + 'PP: '+ fround(maxPrice) + ',CP:'+ str(currentPriceAPICall) + ',' +'QTY: '+ str(numberOfStocks) +', AS: '+ fround(amountSpent) +','+ 'CV: '+ currentValueAPICall + ',ROI: ' + fround(ROI))
                                dfStocksCurrentlyBought = dfStocksCurrentlyBought.append({'STOCK':STOCK , 'PURCHASEDATE' : DATE , 'PURCHASEPRICE': maxPrice , 'QTY': numberOfStocks , 'CURRVALUE': currentValueAPICall ,'CURRPRICE': currentPriceAPICall , 'MAXPRICE' : maxPrice, 'ROC': ROI , 'ACTIVE': 'X' }, ignore_index=True)
                                totalSpent = totalSpent + amountSpent
                                break
    else:
        print('NO RECOMMENDATIONS AS INDEX IS GOING DOWN : ' + str(start_date))


# Update MAX Price of this week


    start_date += delta

weekValueAPICall = 0
weektotalValue = 0
totalValueWeek = 0
totalValueWeekAPICall = 0

print('CHECKING ROI')

stockWeekList = dfStocksCurrentlyBought.query("ACTIVE=='X'").STOCK.unique()
for stockWeekValue in stockWeekList:
    queryGetCurrentStockDataWeekValue = 'STOCK' + '=='+"'"+stockWeekValue+"'"
    dfCurrentstockWeekValue=dfstocks.query(queryGetCurrentStockDataWeekValue)

    queryGetCurrentPriceWithDateWeekValue="DATE"+"=="+"'"+processingDate+"'"
    dfCurrentstockWeekValue=dfCurrentstockWeekValue.query(queryGetCurrentPriceWithDateWeekValue)
    dfCurrentstockWeekValue=dfCurrentstockWeekValue.tail(1)


    #SUMMARY
    if(dfCurrentstockWeekValue.shape[0])>0:
        priceForWeek=float(dfCurrentstockWeekValue['PRICE'].iloc[0])
        dqQty = dfStocksCurrentlyBought.query(queryGetCurrentStockDataWeekValue)
        weekQty=float(dqQty['QTY'].iloc[0])
        stockValue=float(priceForWeek*weekQty)
        weektotalValue = float(weektotalValue + stockValue)

        weekPriceAPICall = 0
#        try:
#            weekPriceAPICall = yft.Ticker(stockWeekValue).info['regularMarketPrice']
        queryGetCurrentStockPriceWeek = 'STOCK' + '==' + "'" + stockWeekValue + "'"
        dfCurrentStockPriceWeek = dfCurrentPrice.query(queryGetCurrentStockPriceWeek)
        if(not dfCurrentStockPriceWeek.empty):
            weekPriceAPICall = float(dfCurrentStockPriceWeek['CURRPRICE'].iloc[0])
            weekValueAPICall = weekPriceAPICall * weekQty
            weektotalValueAPI = float(weektotalValueAPI) + float(weekValueAPICall)
            print("Check stock " + str(stockWeekValue) + ' getting ' + fround(weekValueAPICall) + 'and total is ' + fround(weektotalValueAPI) )

weektotalValueAPI = weektotalValueAPI + totalStopLossMoney
weektotalValue = weektotalValue + totalStopLossMoney
if( weektotalValue > 0):
    currWeekROI = fround((weektotalValue -totalSpent)/totalSpent*100)
    currPriceAPI = fround((weektotalValueAPI -totalSpent)/totalSpent*100)



printtotalSpent = fround(totalSpent)
printtotalBalance = fround(weektotalValueAPI)
printProfitpercentTotal = fround(((weektotalValueAPI - totalSpent ) / totalSpent ) * 100)

printSpentLosgintStocks = fround(moneySpentForLosingStocks)
printTotalObtainedLosingStocks = fround(totalStopLossMoney)

printProfitpercentLoss = 0
if(not moneySpentForLosingStocks==0):
    printProfitpercentLoss = fround((( totalStopLossMoney - moneySpentForLosingStocks ) / moneySpentForLosingStocks ) * 100)
else:
    printProfitpercentLoss=fround(0)

printSpentWinningStocks = fround(totalSpent -moneySpentForLosingStocks )
printGotFromWinningStocks = fround( weektotalValueAPI -totalStopLossMoney )
printProfitpercentWin = fround((( float(printGotFromWinningStocks) - float(printSpentWinningStocks) ) / float(printSpentWinningStocks) ) * 100)

print('Total spent : ' +printtotalSpent)
print('Total value : ' +printtotalBalance)


print('Total spent for losing stocks : ' + printSpentLosgintStocks )
print('Total got from liquidating losing stocks : ' + printTotalObtainedLosingStocks )

print('TOtal spent for winning stocks : ' + printSpentWinningStocks )
print('TOtal got for winning stocks : ' + printGotFromWinningStocks )

print('Overall profit %: ' + printProfitpercentTotal  + " Win profit %: " + printProfitpercentWin + " Loosing stocks loss %: " +printProfitpercentLoss )

print('Spent:'+fround(totalSpent)+' '+'WeekValue:' +processingDate+' '+ 'CurrPriceWeekValue:' +fround(weektotalValueAPI) +' CURRROI%='+ currPriceAPI +'%' )
outputTotalLoss=moneySpentForLosingStocks+totalStopLossMoney
outputStocksMakingLoss=overAllStocksPurchased-totalStocksHeld
print(str(start_date_print) + ' : ' + str(end_date_print) + ' : ' + str(c_stoplossfactorindexdown)+' : '+str(c_stoplossindexup)+' : '+str(overAllStocksPurchased)+' : '+str(totalStocksHeld)+' : '+str(outputStocksMakingLoss)+' : '+fround(outputTotalLoss))
outputlosspercent=0
if(not moneySpentForLosingStocks==0):
    outputlosspercent=((moneySpentForLosingStocks - totalStopLossMoney)/moneySpentForLosingStocks*100)
outputprofitcercentage=(weektotalValueAPI-totalStopLossMoney)/(totalSpent-moneySpentForLosingStocks)*100
print('Loss %'+fround(outputlosspercent)+' Profit %'+fround(outputprofitcercentage))

print("--- %s seconds ---" % (time.time() - start_time))






Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Index : 2019-09-06 - 1
1 ++ : wen,2019-09-06,PP: 21.88,CP:21.55,QTY: 114, AS: 2494.21,CV: 2456.7,ROI: 45.8
Index : 2019-10-04 - 0
NO RECOMMENDATIONS AS INDEX IS GOING DOWN : 2019-10-04
Index : 2019-11-01 - 1
2 ++ : bjri,2019-11-01,PP: 43.21,CP:39.94,QTY: 57, AS: 2462.91,CV: 2276.58,ROI: 41.2
Index : 2019-11-29 - 1
3 ++ : splk,2019-11-29,PP: 140.5,CP:161.49,QTY: 17, AS: 2388.5,CV: 2745.33,ROI: 74.75
Index : 2019-12-27 - 1
4 ++ : anss,2019-12-27,PP: 258.89,CP:269.2,QTY: 9, AS: 2330.01,CV: 2422.8,ROI: 69.82
CHECKING ROI
Check stock wen getting 2456.7and total is 2456.7
Check stock bjri getting 2276.58and total is 4733.28
Check stock splk getting 2745.33and total is 7478.61
Check stock anss getting 2422.8and total is 9901.41
Total spent : 9675.63
Total value : 9901.41
Total spent for losing stocks : 0
Total got from liquidating losing stocks : 0
TOtal spent for w